In [20]:
using Dates

include("src/macros.jl");
include("src/simulation.jl");
include("src/ops/Func.jl");
include("src/srcs/StreamSource.jl");
include("src/srcs/IterableSource.jl");
include("src/ops/Collect.jl");

In [21]:
data1 = [
    (DateTime("2020-01-01T12:00:00"), 1.1),
    (DateTime("2020-01-01T12:01:00"), 1.0),
    (DateTime("2020-01-01T12:03:00"), 1.3),
    (DateTime("2020-01-01T12:06:00"), 1.6),
    (DateTime("2020-01-01T12:11:00"), 1.2),
    (DateTime("2020-01-01T13:11:00"), 1.7),
    (DateTime("2020-01-01T14:11:00"), 1.8),
]

data2 = [
    (DateTime("2020-01-01T12:03:00"), 11.0),
    (DateTime("2020-01-01T12:04:00"), 11.6),
    (DateTime("2020-01-01T13:09:00"), 11.7),
    (DateTime("2020-01-01T14:10:00"), 11.8),
]

src1 = IterableSource(data1)
src2 = IterableSource(data2)

sink1 = Collect{Float64}()
sink2 = Collect{Float64}()

pipe1 = @streamops Transform(x -> x[2]^2) Transform(x -> x - 1) sink1
pipe2 = @streamops Transform(x -> sqrt(x[2])) sink2

srcs = [src1, src2]
pipes = [pipe1, pipe2]
simulate_chronological_stream(DateTime, srcs, pipes)

display(sink1.out)
display(sink2.out)

7-element Vector{Float64}:
 0.2100000000000002
 0.0
 0.6900000000000002
 1.5600000000000005
 0.43999999999999995
 1.8899999999999997
 2.24

4-element Vector{Float64}:
 3.3166247903554
 3.40587727318528
 3.420526275297414
 3.4351128074635335

### PeriodicSource

In [22]:
include("src/srcs/PeriodicSource.jl");

periodic_source = PeriodicSource(
    DateTime(2017, 1, 1),
    DateTime(2017, 1, 10),
    Day(1);
    inclusive_end=false
)

for _ in 1:12
    println(next!(periodic_source))
end

2017-01-01T00:00:00
2017-01-02T00:00:00
2017-01-03T00:00:00
2017-01-04T00:00:00
2017-01-05T00:00:00
2017-01-06T00:00:00
2017-01-07T00:00:00
2017-01-08T00:00:00
2017-01-09T00:00:00
nothing
nothing
nothing


In [23]:
@code_warntype next!(periodic_source)

MethodInstance for next!(::PeriodicSource{DateTime, Day})
  from next!(source::PeriodicSource) @ Main ~/repos/StreamOps.jl/src/srcs/PeriodicSource.jl:31
Arguments
  #self#::Core.Const(next!)
  source::PeriodicSource{DateTime, Day}
Locals
  date::DateTime
Body::Union{Nothing, DateTime}
1 ─       (date = Base.getproperty(source, :current_date))
│   %2  = Base.getproperty(source, :inclusive_end)::Bool
└──       goto #6 if not %2
2 ─ %4  = date::DateTime
│   %5  = Base.getproperty(source, :end_date)::DateTime
│   %6  = (%4 > %5)::Bool
└──       goto #5 if not %6
3 ─       return Main.nothing
4 ─       Core.Const(:(goto %10))
5 ┄       goto #9
6 ─ %11 = date::DateTime
│   %12 = Base.getproperty(source, :end_date)::DateTime
│   %13 = (%11 >= %12)::Bool
└──       goto #9 if not %13
7 ─       return Main.nothing
8 ─       Core.Const(:(goto %17))
9 ┄ %17 = Base.getproperty(source, :current_date)::DateTime
│   %18 = Base.getproperty(source, :period)::Day
│   %19 = (%17 + %18)::DateTime
│        

### IterableSource

In [24]:
include("src/srcs/IterableSource.jl");

values = collect(1:15)

iterable_source = IterableSource(values)

for _ in 1:length(values)+1
    println(next!(iterable_source))
end

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
nothing


In [25]:
# @code_native next!(iterable_source)
# @code_llvm debuginfo=:none next!(iterable_source)
@code_warntype debuginfo=:none next!(iterable_source)

MethodInstance for next!(::IterableSource{Vector{Int64}})
  from next!(src::IterableSource{D}) where D @ Main ~/repos/StreamOps.jl/src/srcs/IterableSource.jl:19
Static Parameters
  D = Vector{Int64}
Arguments
  #self#::Core.Const(next!)
  src::IterableSource{Vector{Int64}}
Locals
  val::Int64
  value::Int64
  pos::Int64
Body::Union{Nothing, Int64}
1 ─       Core.NewvarNode(:(val))
│         Core.NewvarNode(:(value))
│         (pos = Base.getproperty(src, :position))
│   %4  = pos::Int64
│   %5  = Base.getproperty(src, :data)::Vector{Int64}
│   %6  = Main.length(%5)::Int64
│   %7  = (%4 >= %6)::Bool
└──       goto #4 if not %7
2 ─       return Main.nothing
3 ─       Core.Const(:(goto %11))
4 ┄ %11 = Base.getproperty(src, :position)::Int64
│   %12 = (%11 + 1)::Int64
│         Base.setproperty!(src, :position, %12)
│         nothing
│   %15 = Base.getproperty(src, :data)::Vector{Int64}
│   %16 = (pos + 1)::Int64
│         (val = Base.getindex(%15, %16))
│         nothing
│         (value 

### DataFrameSource

In [26]:
include("src/srcs/DataFrameRowSource.jl");

df = DataFrame(
    datetime=DateTime.(["2017-01-01", "2017-01-02", "2017-01-03"]),
    value=[1, 2, 3],
    text=["a", "bc", "def"])

output = DataFrameRow[]

# DataFrameRows
df_source = DataFrameRowSource(df)

for _ in 1:nrow(df)+1
    println(next!(df_source))
end

output

DataFrameRow
 Row │ datetime             value  text   
     │ DateTime             Int64  String 
─────┼────────────────────────────────────
   1 │ 2017-01-01T00:00:00      1  a
DataFrameRow
 Row │ datetime             value  text   
     │ DateTime             Int64  String 
─────┼────────────────────────────────────
   2 │ 2017-01-02T00:00:00      2  bc
DataFrameRow
 Row │ datetime             value  text   
     │ DateTime             Int64  String 
─────┼────────────────────────────────────
   3 │ 2017-01-03T00:00:00      3  def
nothing


DataFrameRow[]

In [27]:
@code_warntype next!(df_source)

MethodInstance for next!(::DataFrameRowSource{DataFrame})
  from next!(source::DataFrameRowSource) @ Main ~/repos/StreamOps.jl/src/srcs/DataFrameRowSource.jl:21
Arguments
  #self#::Core.Const(next!)
  source::DataFrameRowSource{DataFrame}
Locals
  val::DataFrameRow{DataFrame, DataFrames.Index}
  row::DataFrameRow{DataFrame, DataFrames.Index}
  pos::Int64
  @_6::DataFrameRow{DataFrame, DataFrames.Index}
Body::Union{Nothing, DataFrameRow{DataFrame, DataFrames.Index}}
1 ─       Core.NewvarNode(:(val))
│         Core.NewvarNode(:(row))
│         (pos = Base.getproperty(source, :position))
│   %4  = pos::Int64
│   %5  = Base.getproperty(source, :df)::DataFrame
│   %6  = Main.size(%5, 1)::Int64
│   %7  = (%4 >= %6)::Bool
└──       goto #4 if not %7
2 ─       return Main.nothing
3 ─       Core.Const(:(goto %11))
4 ┄       nothing
└──       goto #6 if not true
5 ─ %13 = Base.getproperty(source, :df)::DataFrame
│   %14 = (pos + 1)::Int64
│         (@_6 = (view)(%13, %14, Main.:(:)))
└──       g

In [28]:
# include("src/ops/OpTimestamper.jl");

# OpTimestamper{DateTime,DateTime}(; date_fn=x -> x)

### simulate_chronological_stream

In [29]:
include("src/srcs/PeriodicSource.jl");
include("src/srcs/IterableSource.jl");
include("src/ops/Combine.jl");
include("src/ops/Print.jl");
include("src/ops/Func.jl");
include("src/ops/Lag.jl");
include("src/ops/Print.jl");
include("src/macros.jl");
include("src/simulation.jl");

dates_source = IterableSource([
    (DateTime(2017, 1, 2),)
    (DateTime(2017, 1, 5),)
    (DateTime(2017, 1, 10),)
])

numbers_source = IterableSource([
    (DateTime(2017, 1, 1), 1.0),
    (DateTime(2017, 1, 2), 2.0),
    (DateTime(2017, 1, 3), 3.0),
    (DateTime(2017, 1, 5), 4.0),
    (DateTime(2017, 1, 7), 5.0)
])

combiner = @streamops begin
    Combine{Tuple{Int64,DateTime}}(
        2;
        slot_fn=evt -> evt[1]
    )
    # @filter x -> !any(isnothing.(x))
    Transform(x -> println("$(x[1]) | $(x[2])"))
end

pipes = [
    @streamops Transform(x -> (1, x[1])) combiner
    @streamops Transform(x -> (2, x[1])) combiner
]

simulate_chronological_stream(
    DateTime,
    [dates_source, numbers_source],
    pipes)

nothing | (2, DateTime("2017-01-01T00:00:00"))
(1, DateTime("2017-01-02T00:00:00")) | (2, DateTime("2017-01-01T00:00:00"))
(1, DateTime("2017-01-02T00:00:00")) | (2, DateTime("2017-01-02T00:00:00"))
(1, DateTime("2017-01-02T00:00:00")) | (2, DateTime("2017-01-03T00:00:00"))
(1, DateTime("2017-01-05T00:00:00")) | (2, DateTime("2017-01-03T00:00:00"))
(1, DateTime("2017-01-05T00:00:00")) | (2, DateTime("2017-01-05T00:00:00"))
(1, DateTime("2017-01-05T00:00:00")) | (2, DateTime("2017-01-07T00:00:00"))
(1, DateTime("2017-01-10T00:00:00")) | (2, DateTime("2017-01-07T00:00:00"))


In [30]:
@code_warntype simulate_chronological_stream(DateTime, [periodic_source, dates_source, numbers_source])
# @code_lowered simulate_chronological_stream(timestamped_sources)

In [31]:
using Statistics

include("src/ops/Combine.jl");
include("src/ops/Aggregate.jl");
include("src/macros.jl");

prices_source = IterableSource(
    map(x -> (DateTime(2020, 1, 1) + Minute(x), 1e2randn()), 0:9:60)
)

volumes_source = IterableSource(
    map(x -> (DateTime(2020, 1, 1) + Minute(x), 1e4randn()), 0:13:90)
)

volatility_source = IterableSource(
    map(x -> (DateTime(2020, 1, 1) + Minute(x), 5e-3randn()), 0:7:40)
)

combiner = @streamops begin
    Combine{Tuple{Int64,DateTime,Float64}}(
        3;
        slot_fn=x -> x[1],
        combine_fn=x -> begin
            map(x -> isnothing(x) ? x : (x[2], x[3]), x)
        end
    )
    @filter x -> !any(isnothing.(x))
    Aggregate{Vector{Tuple{DateTime,Float64}},DateTime}(;
        key_fn=x -> round(maximum(map(x -> x[1], x)), Dates.Minute(5), RoundUp),
        agg_fn=(period, buffer) -> begin
            (period, map(x -> x[2], last(buffer)))
        end
    )
    @filter !isnothing
    Print()
end

pipes = [
    @streamops Transform(x -> (1, x[1], x[2])) combiner
    @streamops Transform(x -> (2, x[1], x[2])) combiner
    @streamops Transform(x -> (3, x[1], x[2])) combiner
]

simulate_chronological_stream(
    DateTime,
    [prices_source, volumes_source, volatility_source],
    pipes
)

(DateTime("2020-01-01T00:00:00"), [-100.90481382416148, 3824.203790032726, -0.005360887347502435])
(DateTime("2020-01-01T00:10:00"), [36.979465096715025, 3824.203790032726, 0.0033378296461676195])
(DateTime("2020-01-01T00:15:00"), [36.979465096715025, -7025.506322542477, -0.0017958328521890602])
(DateTime("2020-01-01T00:20:00"), [16.580684473146007, -7025.506322542477, -0.0017958328521890602])
(DateTime("2020-01-01T00:25:00"), [16.580684473146007, -7025.506322542477, -0.0009147233139117385])
(DateTime("2020-01-01T00:30:00"), [-158.28611404264166, 5818.94513524999, -0.002148225414709444])
(DateTime("2020-01-01T00:35:00"), [-158.28611404264166, 5818.94513524999, 0.010174361298481663])
(DateTime("2020-01-01T00:40:00"), [16.204620301323025, 12594.659789890926, 0.010174361298481663])
(DateTime("2020-01-01T00:45:00"), [192.37819751651637, 12594.659789890926, 0.010174361298481663])
(DateTime("2020-01-01T00:55:00"), [-31.333616265803034, -6970.5142515728, 0.010174361298481663])
(DateTime("2020